# DataExample

In [17]:
import pandas as pd
from sklearn.metrics import average_precision_score, roc_auc_score

def show_scores(df):
    print("roc_auc_score",roc_auc_score(y_true=df.ytrue, y_score=df.ypred))
    print("average_precision_score", average_precision_score(df.ytrue, df.ypred))    

In [2]:
panpepdir = "/Users/kyoheikoyama/workspace/PanPep"

In [3]:
# !wc ./Data/majority_testing_dataset.csv

In [4]:
pd.read_csv(f"{panpepdir}/Data/Example_majority.csv")['Label'].value_counts()

1          11616
0          11616
Unknown     5230
Name: Label, dtype: int64

In [5]:
!head {panpepdir}/Data/Example_majority.csv

Peptide,CDR3,Label
ATDALMTGY,CAISESQGNTEAFF,1
ATDALMTGY,CAISEDRALVSYTF,1
ATDALMTGY,CAISEDRALNEQFF,1
ATDALMTGY,CAVQPGQGMQPQHF,1
ATDALMTGY,CAISEGAMGNQPQHF,1
ATDALMTGY,CAISDQTSGNQPQHF,1
ATDALMTGY,CAISDAGAGNQPQHF,1
ATDALMTGY,CAISEGTGGHQPQHF,1
ATDALMTGY,CAISESSAGNQPQHF,1


# Process Data

In [6]:
df1 = pd.read_parquet('../data/entire_data.parquet')
df1['Label'] = df1['sign'].values

df2 = pd.read_parquet("../data/recent_data_test_entire_crossatten.parquet")
df2['Label'] = "Unknown"
df0 = pd.concat([df2, df1])

df0 = df0.drop_duplicates(subset=['peptide','tcrb'])
df0['Peptide'] = df0['peptide']
df0['CDR3'] = df0['tcrb']
# df0['Label'] = df0['sign']
df0 = df0.sort_values(by=['peptide', 'tcrb', 'sign'])
df0[["CDR3","Label","Peptide"]].to_csv("../../PanPep/Data/recent-test-withTrain.csv", index=None)

In [7]:
df0.Label.value_counts()

0          90848
Unknown    30221
1          13511
Name: Label, dtype: int64

In [10]:
df0 = pd.read_parquet("../data/recent_data_test_entire_crossatten.parquet")
df0 = df0.drop_duplicates(subset=['peptide','tcrb'])
df0['Peptide'] = df0['peptide']
df0['CDR3'] = df0['tcrb']
df0['Label'] = df0['sign']
df0 = df0.sort_values(by=['peptide', 'tcrb', 'sign'])
df0[["CDR3","Label","Peptide"]].to_csv("../../PanPep/Data/recent-test.csv", index=None)

In [11]:
!head ../../PanPep/Data/recent-test.csv

CDR3,Label,Peptide
CAISDLTGEIYSGYTF,0,AAFKRSCLK
CASSFGGSRLGEETQYF,0,AAFKRSCLK
CASSFGQRDRGYWRNTEAFF,0,AAFKRSCLK
CASSHPEGDVLTF,0,AAFKRSCLK
CASSKTAGWEYGYTF,0,AAFKRSCLK
CASSLILAGLNQPQHF,0,AAFKRSCLK
CASSLTSGSEQFF,0,AAFKRSCLK
CASSNPGPWGSPLHF,0,AAFKRSCLK
CASSPIKDTRQEYTGELFF,0,AAFKRSCLK


In [12]:
!wc ../../PanPep/Data/recent-test.csv

   30222   30222  853306 ../../PanPep/Data/recent-test.csv


In [13]:
path = "../../PanPep/Data/recent-test.csv"
df = pd.read_csv(path)
df = df.sort_values(by=['Peptide','CDR3'])
df.to_csv(path,index=None)

In [14]:
"No duplicates of (pep, cdr3) = ", len(df)==len(df.drop_duplicates(subset=['Peptide','CDR3'])) #.Label.value_counts()

('No duplicates of (pep, cdr3) = ', True)

In [15]:
# df.Peptide.value_counts(), df.CDR3.value_counts()

# Please run these in the Panpep directory

## Test set output

In [12]:
path = "../../PanPep/Output/recent-test_output.csv"
df_panpep_recenttest = pd.read_csv(path)

df_panpep_recenttest = pd.merge(df_panpep_recenttest, df[['Peptide', 'CDR3', 'Label']],
         how='left'
        )

df_panpep_recenttest['ytrue'] = df_panpep_recenttest.Label
df_panpep_recenttest['ypred'] = df_panpep_recenttest.Score

In [13]:
show_scores(df_panpep_recenttest)

roc_auc_score 0.49139034908520973
average_precision_score 0.17123114269525874


In [14]:
df_panpep_recenttest.columns

Index(['Peptide', 'CDR3', 'Score', 'Label', 'ytrue', 'ypred'], dtype='object')

In [15]:
df_panpep_recenttest[['Peptide', 'CDR3','ytrue', 'ypred']]\
.to_parquet("../../TCRPrediction/data/recent_data_test_PanPep.parquet")

In [19]:
path = "../../PanPep/Output/recent-test_output-fewshot.csv"

df_panpep_recenttest = pd.read_csv(path)
df_panpep_recenttest = pd.merge(df_panpep_recenttest, df[['Peptide', 'CDR3', 'Label']],
         how='left'
        )

df_panpep_recenttest['ytrue'] = df_panpep_recenttest.Label
df_panpep_recenttest['ypred'] = df_panpep_recenttest.Score

show_scores(df_panpep_recenttest)

roc_auc_score 0.5336874870031201
average_precision_score 0.18967045761441126


In [20]:
path = "../../PanPep/Output/recent-test_output-majority.csv"

df_panpep_recenttest = pd.read_csv(path)
df_panpep_recenttest = pd.merge(df_panpep_recenttest, df[['Peptide', 'CDR3', 'Label']],
         how='left'
        )

df_panpep_recenttest['ytrue'] = df_panpep_recenttest.Label
df_panpep_recenttest['ypred'] = df_panpep_recenttest.Score

show_scores(df_panpep_recenttest)

roc_auc_score 0.5336874870031201
average_precision_score 0.18967045761441126


In [21]:
# df_panpep_recenttest[['Peptide', 'CDR3','ytrue', 'ypred']]\
# .to_parquet("../../TCRPrediction/data/recent_data_test_PanPep.parquet")

In [28]:
temp = df_panpep_recenttest.sort_values(by=['ytrue'], ascending=False)

temp = pd.concat([temp.query('ytrue==1'), 
                  temp.query('ytrue!=1').sample(temp.ytrue.sum(), random_state=0)
                 ])

In [30]:
show_scores(temp)

roc_auc_score 0.5358552558110228
average_precision_score 0.5253360409793577


## Model including the entire-data training as input csv

In [64]:
path = "../../PanPep/Output/recent-test_output-majority-withTrain.csv"

df_panpep_recenttest = pd.read_csv(path)
df_panpep_recenttest = pd.merge(df_panpep_recenttest, df0.query('Label=="Unknown"')[['sign','Peptide', 'CDR3', 'Label']],
         how='right'
        )

df_panpep_recenttest['ytrue'] = df_panpep_recenttest.sign
df_panpep_recenttest['ypred'] = df_panpep_recenttest.Score
df_panpep_recenttest['ypred'] = df_panpep_recenttest['ypred'].apply(process)
print(len(df_panpep_recenttest))
show_scores(df_panpep_recenttest)

30221
roc_auc_score 0.49139166416649194
average_precision_score 0.17122693558591198


In [65]:
path = "../../PanPep/Data/zero_dataset.csv"

df_zeroshot = pd.read_csv(path)

In [67]:
df_zeroshot.label.value_counts()

1    857
Name: label, dtype: int64

In [70]:
df_zeroshot.peptide.value_counts()

VQYLGMFPV      4
RPHERNGFTV     4
EENLLDFVRF     4
YLLFMIGYT      4
RMAAISNTV      4
              ..
TYDTVHRHL      1
VLDLFQGQL      1
HIMPDTAEI      1
TLYSLTLLY      1
LAITPEIAPYF    1
Name: peptide, Length: 491, dtype: int64